In [1]:
import tkinter as tk
from tkinter import ttk
import socket
import threading
import ast
from Techniques.des_encry_decry import bin_to_hexa,encrypt,pad,decrypt
from Techniques.rc4 import rc4_encrypt,rc4_decrypt
from Difii_Hellman import read
from Techniques.Elgammal import elg_encrypt,elg_decrypt
from Techniques.RSA import generate_key,RSA_encrypt,RSA_decrypt
from Techniques.AES import aes_decrypt,matrix_to_hex,hex_to_matrix,aes_encrypt

In [2]:
class CryptoChatClient:
    def __init__(self, root):
        self.root = root
        self.root.title("Client")

        self.crypto_technique_var = tk.StringVar()

        # Create a dropdown menu for crypto techniques
        crypto_label = tk.Label(root, text="Crypto Technique:")
        crypto_label.grid(row=0, column=0, padx=10, pady=10, sticky="e")

        crypto_options = ["AES", "RSA", "DES", "RC-4", "ELGAMMAL"]  # Add more as needed
        crypto_dropdown = ttk.Combobox(root, textvariable=self.crypto_technique_var, values=crypto_options)
        crypto_dropdown.grid(row=0, column=1, padx=10, pady=10, sticky="w")
        crypto_dropdown.set(crypto_options[0])  # Set default value

        # Create a text box for messages
        message_label = tk.Label(root, text="Message:")
        message_label.grid(row=1, column=0, padx=10, pady=10, sticky="e")

        self.message_entry = tk.Entry(root, width=40)
        self.message_entry.grid(row=1, column=1, padx=10, pady=10, sticky="w")

        # Create a chat window
        self.chat_window = tk.Text(root, height=15, width=50, state=tk.DISABLED)
        self.chat_window.grid(row=2, column=0, columnspan=2, padx=10, pady=10)

        # Create a send button
        send_button = tk.Button(root, text="Send", command=self.send_message)
        send_button.grid(row=3, column=0, columnspan=2, pady=10)

        # Initialize the socket (without connecting)
        self.my_socket = socket.socket()
        self.keys = read()
        print(self.keys)
        # Start a thread to listen for incoming connections
        threading.Thread(target=self.connector_thread).start()

    def connector_thread(self):
        try:
            # Connect the socket when needed
            self.my_socket.connect(("localhost", 4349))
            print("Connected to server")

            # Start a thread to listen for incoming messages
            threading.Thread(target=self.receive_messages).start()

        except ConnectionError as e:
            print(f"Connection error: {e}")

    def receive_messages(self):
        while True:
            try:
                data = self.my_socket.recv(1024).decode('utf-8')
                
                if not data:
                    break

                crypto_technique, message = data.split(":", 1)     
                     
                # Perform crypto operations on the received message using the selected technique
                if crypto_technique == "DES":
                    # Add decryption logic here
                    key= str(self.keys['bob_shared_key'])
                    key = pad(key) 
                    decrypted_message = bin_to_hexa(decrypt(message,key))
                    self.display_message(f"{crypto_technique}: {decrypted_message}")
                if crypto_technique == "RC-4":
                    key= str(self.keys['bob_shared_key'])
                    decrypted =rc4_decrypt(key, message)
                    self.display_message(f"{crypto_technique}: {decrypted}")
                if crypto_technique == "ELGAMMAL":
                    c1,c2 = message.split(",",1)
                    print(c1,":",c2)
                    p = self.keys['p']
                    q = self.keys['q']
                    bob_priv = self.keys['bob_private_key']
                    decrypted =elg_decrypt(p,q,c1,c2,bob_priv)
                    self.display_message(f"{crypto_technique}: {decrypted}")

                if crypto_technique == "RSA":
                    p = self.keys['p']
                    q = self.keys['q']
                    fn = self.keys['fn']
                    e = self.keys['e']
                    my_list = eval(message)                    
                    pub , priv = generate_key(p,q,fn,e)
                    decrypted = RSA_decrypt(my_list,priv)
                    self.display_message(f"{crypto_technique}: {decrypted}")

                if crypto_technique == 'AES':
                    key_hex = '8A2F64BC3E7D1A9F5C0E87B2D91FE4CA'
                    key_matrix = hex_to_matrix(key_hex)
                    matrix = ast.literal_eval(message)
                    decrypted_message = aes_decrypt(matrix, key_matrix)

                    decrypted_hex = matrix_to_hex(decrypted_message)
                    self.display_message(f"{crypto_technique}: {decrypted_hex}")

            except ConnectionResetError:
                print("Connection closed.")
                break

    def send_message(self):
        try:          
            message = self.message_entry.get()
            crypto_technique = self.crypto_technique_var.get()

            # Perform crypto operations on the message using the selected technique
            if crypto_technique == "DES":
                plain_text=pad(message)            
                key= str(self.keys['bob_shared_key'])
                key=pad(key)
                cipher_text=bin_to_hexa(encrypt(plain_text,key))
                self.display_message(f"{crypto_technique}: {cipher_text}")
                
            if crypto_technique == "RC-4":
                key= str(self.keys['bob_shared_key'])
                cipher_text = rc4_encrypt(key, message)
                self.display_message(f"{crypto_technique}: {cipher_text.hex()}")

            if crypto_technique == "AES":
                
                key_hex= '8A2F64BC3E7D1A9F5C0E87B2D91FE4CA'
                text_hex= message
                text_matrix = hex_to_matrix(text_hex)
                key_matrix = hex_to_matrix(key_hex)
                cipher_text=aes_encrypt(text_matrix,key_matrix)
                print(cipher_text)
                self.display_message(f"{crypto_technique}: {cipher_text}")

            if crypto_technique == "ELGAMMAL":
                p = self.keys['p']
                q = self.keys['q']
                g = self.keys['gg']
                pub_Bob = self.keys['bob_public_key']
                c1,c2 = elg_encrypt(p,q,g,pub_Bob,int(message))
                cipher_text = f'{c1},{c2}'
                self.display_message(f"{crypto_technique}: {cipher_text}")

            if crypto_technique == "RSA":
                p = self.keys['p']
                q = self.keys['q']
                fn = self.keys['fn']
                e = self.keys['e']
                pub , priv = generate_key(p,q,fn,e)
                cipher_text = RSA_encrypt(message,pub)
                self.display_message(f"{crypto_technique}: {cipher_text}")
            # Send the message to the client
            data = f"{crypto_technique}:{cipher_text}"
            msg = bytes(data, 'utf-8')
            self.my_socket.send(msg)

        except ConnectionError as e:
            print(f"Connection error: {e}")

    def display_message(self, message):
        self.chat_window.config(state=tk.NORMAL)
        self.chat_window.insert(tk.END, message + "\n")
        self.chat_window.config(state=tk.DISABLED)

if __name__ == "__main__":
    root = tk.Tk()
    app = CryptoChatClient(root)
    root.mainloop()

{'p': 6983, 'q': 7369, 'n': 51457727, 'g': 6455005, 'gg': 3884, 'e': 8228759, 'fn': 51443376, 'alice_private_key': 37627137, 'bob_private_key': 3006957, 'alice_public_key': 28524543, 'bob_public_key': 12807914, 'alice_shared_key': 51211765, 'bob_shared_key': 51211765}
Connected to server
Connection closed.
